#### Organazing files for generate predictions

Inside the folde `test_video_list_and_name_mapping` a `test_masks` subfolder is generating that will contain the predictions of each frame per camera. for every time and camera will be a folder and a text file; one contain mask corresponding to that frame and text file contain relavite path to the mask ans scores

```
171206_034157032_Camera_5
171206_034157032_Camera_5_instanceIds.txt
```

In [2]:
import glob

In [181]:
mds = glob.glob("../data/test/list_test_mapping/*.txt")
results = pd.read_csv("../result.csv")
test_r =  pd.read_csv("../data/test_results.csv")
masks = glob.glob("./test_masks/*.jpg")

my_result = pd.read_csv("../my_csv.csv")
images_ids = results.ImageId.unique().tolist()

In [87]:
ss=pd.read_csv(mds[8], sep='\t', header=None)

In [88]:
ss.head()

,0,1
0,5c90ef0b878a1e5a3e0d57a8b08bd066,road02_ins\ColorImage\Record007\Camera 5\17092...
1,7912a5160c1fbebc3dfcfd88be43b658,road02_ins\ColorImage\Record007\Camera 5\17092...
2,99d5dc333e340680574abe8c40f3392d,road02_ins\ColorImage\Record007\Camera 5\17092...
3,974be7bb825c6f4b8c38c3b8ca6d0f29,road02_ins\ColorImage\Record007\Camera 5\17092...
4,b553b5d1187925cdfc1101e61b8aeade,road02_ins\ColorImage\Record007\Camera 5\17092...


 This organize masks instances into folders per frame

In [89]:
for i in range(len(mds)):
    frames = pd.read_csv(mds[i], sep='\t', header=None)
    for j in range(frames.shape[0]):
        image_id = frames.iloc[j][0]
        path = frames.iloc[j][1].split("\\")[-1].split(".")[-2]
        preds = glob.glob("./test_masks/"+image_id+"*jpg")
        if os.path.exists("./test_masks/"+path):
            print("Warning already exists: ", path)

        else:
            os.makedirs("./test_masks/"+path)
        for p in preds:
            shutil.move(p, "./test_masks/"+path+"/"+p.split("/")[-1] )


                                                

Warning already exists:  170908_085500854_Camera_6
Warning already exists:  171206_030116480_Camera_6
Warning already exists:  170908_072530085_Camera_6
Warning already exists:  171206_033959261_Camera_5
Warning already exists:  171206_033007333_Camera_5
Warning already exists:  170927_063815166_Camera_6
Warning already exists:  171206_033811429_Camera_6
Warning already exists:  171206_032801900_Camera_5
Warning already exists:  170927_064544406_Camera_5
Warning already exists:  170908_064917249_Camera_5
Warning already exists:  170908_082410385_Camera_6
Warning already exists:  170908_061426186_Camera_5


The cells below generate instances files containing relative path and scores for every mask
of given instance

In [110]:
dirs=next(os.walk("./test_masks/"))[1]

In [166]:
groups = [ sorted(glob.glob("./test_masks/"+dir_+"/*.jpg")) for dir_ in dirs  ] 
counts = [len(x) for x in groups if len(x) == 0]
print( len(counts) )
print(counts)


81
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [174]:
results = pd.read_csv("../data/new_results.csv")

In [277]:
def process_group(group_files, results):

    mid = group_files[0].split("/")[-1].split(".")[-2].split("_")[-2]
    image_set = group_files[0].split("/")[2]

    rr = results[results["ImageId"]== mid].sort_values(by=['mask_file'])
    r_index = rr.index
    with open('./test_masks/'+image_set+"_instanceIds.txt", 'a') as f:
        for  i in range( len(group_files) ):
            im = cv2.imread( group_files[i], cv2.IMREAD_UNCHANGED )
            pixel_count = (im > 0).sum()
            rel_path = group_files[i].replace("./test_masks/", "")

            f.write( "{} {} {}\n".format(rel_path, rr['LabelId'][r_index[i]], rr['Confidence'][r_index[i]]) )
        f.close()

In [278]:
process_group(groups[0], my_result)


In [279]:
pool = Pool(8)
jobs = []
for i in range(len(groups)):
    job = pool.apply_async(process_group, (groups[i], my_result) )
    jobs.append(job)

